In [1]:
import pandas as pd

train = pd.read_csv('data/train_to_colab.csv')

In [2]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train['rendimiento_global'] = le.fit_transform(train['rendimiento_global'])

In [3]:
df = train.copy()

## Extraer año y semestre del periodo
df['periodo_academico'] = df['periodo_academico'].astype(int)

df['anio'] = df['periodo_academico'] // 10
df['semestre'] = df['periodo_academico'] % 10

## Features agregados por programa académico
program_group = df.groupby('e_prgm_academico')

df['prog_mean_indicador_1'] = df['e_prgm_academico'].map(program_group['indicador_1'].mean())
df['prog_mean_indicador_2'] = df['e_prgm_academico'].map(program_group['indicador_2'].mean())
df['prog_mean_indicador_3'] = df['e_prgm_academico'].map(program_group['indicador_3'].mean())
df['prog_mean_indicador_4'] = df['e_prgm_academico'].map(program_group['indicador_4'].mean())

df['prog_mean_rend'] = df['e_prgm_academico'].map(program_group['rendimiento_global'].mean())

## Features agregados por departamento
dept_group = df.groupby('e_prgm_departamento')

df['dept_mean_indicador'] = df['e_prgm_departamento'].map(
    dept_group[['indicador_1','indicador_2','indicador_3','indicador_4']].mean().mean(axis=1)
)

df['dept_mean_rend'] = df['e_prgm_departamento'].map(
    dept_group['rendimiento_global'].mean()
)

## Indicador promedio y dispersión
df['indicador_promedio'] = df[['indicador_1','indicador_2','indicador_3','indicador_4']].mean(axis=1)
df['indicador_std'] = df[['indicador_1','indicador_2','indicador_3','indicador_4']].std(axis=1)

## Ratios relevantes
# Relaciones clave según correlaciones
df['ratio_1_2'] = df['indicador_1'] / (df['indicador_2'] + 1)
df['ratio_1_3'] = df['indicador_1'] / (df['indicador_3'] + 1)
df['ratio_1_4'] = df['indicador_1'] / (df['indicador_4'] + 1)

# Índice consolidado de indicadores correlacionados
df['indicador_234_mean'] = df[['indicador_2','indicador_3','indicador_4']].mean(axis=1)
df['indicador_234_std'] = df[['indicador_2','indicador_3','indicador_4']].std(axis=1)

# Ratio entre indicador independiente e índice
df['ratio_1_234'] = df['indicador_1'] / (df['indicador_234_mean'] + 1)

# Diferencias (importante porque 2–3–4 se mueven juntos)
df['diff_2_3'] = df['indicador_2'] - df['indicador_3']
df['diff_2_4'] = df['indicador_2'] - df['indicador_4']
df['diff_3_4'] = df['indicador_3'] - df['indicador_4']



## Interacciones importantes socioeconómicas × desempeño
df['promedio_x_estrato'] = df['indicador_promedio'] * df['f_estratovivienda']
df['promedio_x_internet'] = df['indicador_promedio'] * df['f_tieneinternet']
df['promedio_x_computador'] = df['indicador_promedio'] * df['f_tienecomputador']

df['horas_x_indicadores'] = df['e_horassemanatrabaja'] * df['indicador_promedio']
df['matricula_x_indicadores'] = df['e_valormatriculauniversidad'] * df['indicador_promedio']


In [4]:
# ===================================================
# IMPORTS
# ===================================================
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier

# ===================================================
# 0. LIMPIEZA Y FIX DE TIPOS
# ===================================================

numeric_fix_cols = [
    'e_valormatriculauniversidad',
    'e_horassemanatrabaja',
    'f_estratovivienda',
    'f_tieneinternet',
    'f_educacionpadre',
    'f_tienelavadora',
    'f_tieneautomovil',
    'e_pagomatriculapropio',
    'f_tienecomputador',
    'f_educacionmadre'
]

# Convertir numéricas que vienen como string
for col in numeric_fix_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Rellenar faltantes
df[numeric_fix_cols] = df[numeric_fix_cols].fillna(-1)

# ===================================================
# 1. DEFINIR CAT_FEATURES
# ===================================================
cat_features = [
    'e_prgm_academico',
    'e_prgm_departamento',
    'f_estratovivienda',
    'f_tieneinternet',
    'f_educacionpadre',
    'f_tienelavadora',
    'f_tieneautomovil',
    'f_tienecomputador',
    'f_educacionmadre'
]

# ===================================================
# 2. DEFINIR TARGET Y FEATURES
# ===================================================
target = 'rendimiento_global'
cols_leakage = ['indicador_1','indicador_2','indicador_3','indicador_4']

features = [c for c in df.columns if c not in [target] + cols_leakage]

X = df[features].copy()
y = df[target].copy()

# ===================================================
# 3. TRAIN / VALID SPLIT
# ===================================================
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# ===================================================
# 4. PREPROCESADOR (OneHot para categóricas)
# ===================================================
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features),
        ("num", "passthrough", [c for c in features if c not in cat_features]),
    ]
)

# ===================================================
# 5. MODELO XGBOOST (CON GPU)
# ===================================================
xgb_model = XGBClassifier(
    n_estimators=800,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="mlogloss",
    objective="multi:softmax",
    tree_method="hist",   # <- Cambiado de 'gpu_hist' a 'hist' para CPU
    # predictor="gpu_predictor", # <- Eliminado predictor de GPU
    random_state=42
)

# Pipeline final
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', xgb_model)
])

# Entrenar
pipeline.fit(X_train, y_train)

# ===================================================
# 6. MÉTRICAS
# ===================================================
preds = pipeline.predict(X_valid)
acc = accuracy_score(y_valid, preds)

print("\n========================================")
print("ACCURACY VALID:", acc)
print("========================================\n")

# ===================================================
# 7. IMPORTANCIA DE FEATURES
# ===================================================
# Para obtener importancias: necesitas obtener las columnas transformadas
ohe = pipeline.named_steps['preprocessor'].named_transformers_['cat']
encoded_cat_names = list(ohe.get_feature_names_out(cat_features))

final_feature_names = encoded_cat_names + [c for c in features if c not in cat_features]

importances = pd.DataFrame({
    'feature': final_feature_names,
    'importance': pipeline.named_steps['model'].feature_importances_
}).sort_values(by='importance', ascending=False)

print(importances.head(25))


ACCURACY VALID: 0.4400649819494585

                                                feature  importance
1027                                     prog_mean_rend    0.025917
1018                        e_valormatriculauniversidad    0.011318
841                        e_prgm_academico_ODONTOLOGIA    0.005734
1029                                     dept_mean_rend    0.005380
957                           e_prgm_departamento_CHOCO    0.005248
1007                               f_educacionmadre_0.0    0.004477
996                                f_educacionpadre_9.0    0.004201
545   e_prgm_academico_LICENCIATURA  EN INGLES Y FRA...    0.004145
1008                               f_educacionmadre_1.0    0.004006
807                           e_prgm_academico_MEDICINA    0.003798
1024                              prog_mean_indicador_2    0.003672
1016                               f_educacionmadre_9.0    0.003581
1020                              e_pagomatriculapropio    0.003544
987        